# Построение движка для игр на pygame - урок 6

## Повторение

После написания первой игры мы можем заметить, что появилось множество методов, вначале цикла которые нам приходится постоянно объявлять в правильном порядке:

In [4]:
from first_engine.game import Game
from first_engine.objects import Object


class FirstGame(Game):

    def run(self):
        hero = Object(parent=self.surface, speed=4, x=100, y=100)
        game_over = False

        while self.RUNNER:
            if not game_over:
                self.fps_counter(60)
                self.window_borders(objects=[hero])
                self.display_update()
                self.window_fill()

                hero.blit()
                hero.motion_control()

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame(width=400, height=400).run()

Давайте объединим их в один метод __cycle_init()__

In [ ]:
"""
Модуль с основным классом-конструктором Game - для создания игры.
"""

import pygame


class Game:
    ...

    def __init__(self, width=400, height=300, color=(0, 0, 0)): ...

    @staticmethod
    def display_update(): ...

    @staticmethod
    def events(): ...

    def window_fill(self): ...

    def fps_counter(self, FPS=30) -> None: ...

    def close(self, event) -> None: ...

    def window_borders(self, objects): ...

    def cycle_init(self, objects: list = None, FPS=30):
        """
        Метод объединяющий в себе все необходимые методы инициализирующие работу главного цикла
        """
        self.fps_counter(FPS)
        if objects:
            self.window_borders(objects=objects)
        self.display_update()
        self.window_fill()

Теперь стартовый код будет выглядеть так:

In [1]:
from first_engine.game import Game
from first_engine.objects import Object


class FirstGame(Game):

    def run(self):
        
        # Здесь описываем переменные и объекты
        
        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init()
                
                # Здесь описываем логику игры

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame().run()

pygame 2.0.1 (SDL 2.0.14, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


Так же на основе нашей первой игры мы поняли, что нужно назначать координаты объектов отдельно, а распологать объект по этим координатам отдельно, поэтому изменим метод __motion()__ на метод __replace()__ и модифицируем его, а так же в методе __motion_control()__ внесем изменения:

In [ ]:
import pygame


class Object:

    def __init__(self, parent: pygame.Surface,
                 width=40, height=40, x=0, y=0,
                 color=(255, 255, 255),
                 speed=1): ...

    def blit(self): ...

    def recolor(self, color): ...

    def __diagonal_speed(self): ...

    def replace(self, x=None, y=None): # раньще был motion()
        if x: # добавим эти условия, что бы можно было назначать произвольное положение объектов, если понадобится
            self.x = x
        if y:
            self.y = y

        self.body.x, self.body.y = self.x, self.y

    def motion_control(self):
        left = pygame.key.get_pressed()[97]
        right = pygame.key.get_pressed()[100]
        up = pygame.key.get_pressed()[119]
        down = pygame.key.get_pressed()[115]

        speed = self.__diagonal_speed() if (left + right + up + down) > 1 else self.speed
        
        # так как тело округляет свои координаты в меньшую сторону,
        # если ему присваивается число с плавающей точкой, то мы будим прибавлять 1 к скорости
        self.x += right * (speed + 1) if speed % 1 != 0 else right * speed
        self.y += down * (speed + 1) if speed % 1 != 0 else down * speed
        self.x -= left * speed
        self.y -= up * speed

        self.replace() # здась заменяем motion() на replace()
        return -left * speed, right * speed, -up * speed, down * speed # возвращаем скорость для каждого направления движения 

    def change_speed(self, speed): ...

## Загрузка спрайтов

Начнем с загрузки фона.

Сначало просмотрим, что из себя представляет само окно, скин персонажей и все что мы можем увидеть глазами. Для этого залезем в документацию метода для создания окон.

In [ ]:
pygame.display.set_mode

In [ ]:
def set_mode(
    size: Optional[Union[Tuple[int, int], Sequence[int]]],
    flags: Optional[int] = 0,
    depth: Optional[int] = 0,
    display: Optional[int] = 0,
    vsync: Optional[int] = 0
) -> Surface: ...

Здесь мы можем видеть все аргументы данного метода, у каждого аргумента есть небольшое описание, например мы можем видеть, что здесь все аргументы принимают целое число.

Так же можно видеть, что все аргументы, кроме первого, имеют значения по умолчанию. Что за что отвечают эти аргументы, можно узнать из интернета, или эмпирическим путем.

Так же, в конце метода, мы можем видеть стрелку -> Surface - это то, что нас сейчас интересует. Это класс - покрытие - это и есть, что мы видем. Мы его уже использовали, что бы создать скин для внутреигровых объектов.

In [ ]:
pygame.Surface(size=(width, height))

Этот класс имеет соответствующие методы, если мы увидем его еще в каких нибудь методах, то будем знать, как его можно использовать.

Теперь рассмотрим метод для загрузки картинок.

In [ ]:
from first_engine.game import Game
from first_engine.objects import Object
import pygame.image


class FirstGame(Game):

    def run(self):
        back_ground = pygame.image.load('./forest.jpg') # метод, для загрузки картинок -> Surfase

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(objects=[hero], FPS=60)
                
                # мы знаем что метод для загрузки картинок возвращает Surfase
                self.surface.blit(back_ground, (0, 0)) # это значит, что мы можем картинкой просто залить фон

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame().run()